In [1]:
!pip install youtube-dl

In [2]:
!pip install --upgrade youtube-dl

Requirement already up-to-date: youtube-dl in c:\users\b-luan\anaconda3\lib\site-packages (2021.3.31)


In [3]:
import youtube_dl
import os
import re
import time


def video_download(link):
    youtube_dl_options = {"writesubtitles":True}

    with youtube_dl.YoutubeDL(youtube_dl_options) as youtube_dl_client:
        youtube_dl_client.download([link])


def audio_download(link):
    afilename = ''
    cmd = ''.join(['youtube-dl -f bestaudio --extract-audio --audio-format mp3 --audio-quality 0 ','"',link, '"'])
    d = os.system(cmd)
    time.sleep(0.5)
    files= os.listdir('./') #得到文件夹下的所有文件名称
    for item in files:
        if item[-3:] == 'm4a' or item[-3:] == 'mp3' :
            afilename = item
    opfile = afilename[:-4] + '.wav'
    # cmd2 = ''.join(['ffmpeg -i ','"', afilename, '" ', '"', opfile, '"'])
    cmd2 = ''.join(['C:/workspace/20210331_SingaporeEnglish/ffmpeg/bin/ffmpeg.exe -i ','"', afilename, '" ', '"', opfile, '"'])
    d = os.system(cmd2)
    return opfile

    
def txt_download(link):
    def post_precessing(filename):
        flg = False
        flg2 = False
        with open(filename, 'r', encoding='utf8') as f:
            source = f.readlines()
        output = filename
        with open(output.replace('vtt','txt'), 'w+', encoding='utf8') as fo:
            for i in range(len(source)):
                if len(source[i]) <= 1:
                    flg2 = True
                if source[i][-4:] == ':0%\n' and flg == False:
                    flg = True
                    continue
                if flg == True and flg2 == True:
                    if (i + 1)%2 == 0:
                        fo.write(source[i])
                        flg = False
                        flg2 = False
                        
    def post_precessing2(filename):
        buff = ''
        with open(filename, 'r', encoding='utf8') as f:
            source = f.readlines()
        output = filename
        with open(output.replace('vtt','txt'), 'w+', encoding='utf8') as fo:
            for line in source:
                if '</c>' not in line and 'position:0%' not in line:
                    new = line.replace(' \n','')
                    if len(new) >=2 and new != buff:
                        fo.write(new)
                        buff = new
    if auto_sub:
        cmd = ''.join(['youtube-dl --write-auto-sub --skip-download ','"',link, '"'])
    else:
        cmd = ''.join(['youtube-dl --write-sub --skip-download ','"',link, '"'])
    # cmd = ''.join(['youtube-dl -f bestaudio --extract-audio --audio-format mp3 --audio-quality 0 ','"',link, '"'])
    
    d = os.system(cmd)
    filename = ''
    # print(os.getcwd())

    filelst = os.listdir('./')
    # print(filelst)
    for file in filelst:
        if file[-3:] == 'vtt':
            filename = file
    if auto_sub:
        post_precessing2(filename)
        os.rename(filename, filename + '.bk')
    return filename


def dataset(textfile, audiofile):
    def output(text):
        with open(audiofile[:-4]+'.dat', 'a+', encoding='utf8') as f:
            f.write(text)
            f.write('\n')
    def text_p_caseA():
        can = []
        timestamp = []
        flg = False
        for i in range(len(resource)):
            line = resource[i].split('\n')[0]
            if line[:2] == "00" and line[0:12] != '00:00:00.000':
                print(line)
                _minute = int(line[3:5])
                _second = int(line[6:8])
                _tail = line[9:12]
                start = '.'.join([str(_minute*60 + _second), _tail])
                print(_minute, _second, _tail, start)
                _minute = int(line[20:22])
                _second = int(line[23:25])
                _tail = line[26:29]
                end = '.'.join([str(_minute*60 + _second), _tail])
                print(_minute, _second, _tail, end)
                if resource[i+1][0].isupper() and flg == False:
                    timestamp.append(start)
                    timestamp.append(end)
                    can.append(resource[i+1].split('\n')[0] + ' ' + resource[i+2].split('\n')[0])
                    flg = True
                elif resource[i+1][0].isupper() and flg == True:
                    op = ''.join(['    ', str(timestamp[0]), '   ', str(timestamp[-1]), ' ', '"', str(' '.join(can)), '"'])
                    print(op)
                    output(op)
                    can = []
                    timestamp = []
                    timestamp.append(start)
                    timestamp.append(end)
                    can.append(resource[i+1].split('\n')[0] + ' ' + resource[i+2].split('\n')[0])
                elif resource[i+1][0].islower() and flg == True:
                    timestamp.append(end)
                    can.append(resource[i+1].split('\n')[0] + ' ' + resource[i+2].split('\n')[0])
                
            #if line[:2] != '00' and line[0] != '\n' and flg:
                #print(line)
            
    print(textfile)
    print(audiofile)
    resource = []
    start = ''
    end = ''
    text = ''
    with open(textfile, 'r', encoding='utf8') as f:
        resource = f.readlines()
    
    
    text_p_caseA()

    

In [7]:
# link = input('Please enter a url link:\n')
# link = 'https://www.youtube.com/watch?v=pJm9Fpb00Bg'
link = 'https://www.youtube.com/watch?v=qaeFnxSfSC4'
res_mode = 'TrainSet' # Txt_Only, Video_Only, Audio_Only, TrainSet
auto_sub = False
op_mode = 'TrainSet' # Debug, Normal, TrainSet
t_filename = ''
a_filename = ''
res = ''

if res_mode == 'Txt_Only':
    t_filename = txt_download(link)
elif res_mode == 'Video_Only':
    video_download(link)
elif res_mode == 'Audio_Only':
    a_filename = audio_download(link)
elif res_mode == 'TrainSet':
    t_filename = txt_download(link)
    a_filename = audio_download(link)
    # t_filename = 'Grit - the power of passion and perseverance _ Angela Lee Duckworth-H14bBuluwB8.en.vtt'
    # a_filename = 'Grit - the power of passion and perseverance _ Angela Lee Duckworth-H14bBuluwB8.wav'
    dataset(t_filename, a_filename)
else:
    pass


TEDxBlue - Angela Lee Duckworth, Ph.D - 10_18_09-qaeFnxSfSC4.en.vtt
TEDxBlue - Angela Lee Duckworth, Ph.D - 10_18_09-qaeFnxSfSC4.wav
00:00:08.677 --> 00:00:12.231
0 8 677 8.677
0 12 231 12.231
00:00:12.231 --> 00:00:16.358
0 12 231 12.231
0 16 358 16.358
    8.677   12.231 "I'm a psychologist and I study achievement. "
00:00:16.358 --> 00:00:20.300
0 16 358 16.358
0 20 300 20.300
    12.231   16.358 "Most psychologists who study achievement study intelligence."
00:00:20.300 --> 00:00:23.235
0 20 300 20.300
0 23 235 23.235
00:00:23.235 --> 00:00:26.445
0 23 235 23.235
0 26 445 26.445
00:00:26.445 --> 00:00:29.913
0 26 445 26.445
0 29 913 29.913
00:00:29.913 --> 00:00:32.438
0 29 913 29.913
0 32 438 32.438
    16.358   29.913 "And if the last talk didn't convince you  and I have a suspicion you didn't need a whole lot of convincing, intelligence is, there's only part of the story,  maybe a very small part of the story. "
00:00:32.438 --> 00:00:34.310
0 32 438 32.438
0 34 310 34.310
00:00

00:04:25.304 --> 00:04:29.610
4 25 304 265.304
4 29 610 269.610
00:04:29.610 --> 00:04:33.068
4 29 610 269.610
4 33 068 273.068
    262.491   269.610 "I would argue that we've done some amount of work on the talents and almost nothing on the unlocking. "
00:04:33.068 --> 00:04:38.257
4 33 068 273.068
4 38 257 278.257
00:04:38.257 --> 00:04:41.888
4 38 257 278.257
4 41 888 281.888
00:04:41.888 --> 00:04:47.716
4 41 888 281.888
4 47 716 287.716
    269.610   281.888 "When I considered what is it that unlocks people's  potential, what enables people to become a world class musician, a world class teacher, a world class performer."
00:04:47.716 --> 00:04:50.178
4 47 716 287.716
4 50 178 290.178
00:04:50.178 --> 00:04:53.590
4 50 178 290.178
4 53 590 293.590
    281.888   290.178 "I struggled with this word to call what I was becoming to understand was one of these key ingredients. "
00:04:53.590 --> 00:04:58.054
4 53 590 293.590
4 58 054 298.054
00:04:58.054 --> 00:05:01.352
4 58 054 298.0

00:10:00.228 --> 00:10:03.004
10 0 228 600.228
10 3 004 603.004
    595.677   600.228 "I finish whatever I begin, I'm determined. "
00:10:03.004 --> 00:10:05.149
10 3 004 603.004
10 5 149 605.149
00:10:05.149 --> 00:10:07.481
10 5 149 605.149
10 7 481 607.481
00:10:07.481 --> 00:10:10.031
10 7 481 607.481
10 10 031 610.031
00:10:10.031 --> 00:10:11.779
10 10 031 610.031
10 11 779 611.779
    600.228   610.031 "I'm going to walk you through a couple of studies,  and then I am going to speculate and it's only going to be  a speculation about what we could possibly do for  young people to cultivate this quality. "
00:10:11.779 --> 00:10:13.564
10 11 779 611.779
10 13 564 613.564
00:10:13.564 --> 00:10:15.137
10 13 564 613.564
10 15 137 615.137
    610.031   613.564 "The first study I want to tell you about was done  at West Point Military Academy. "
00:10:15.137 --> 00:10:17.064
10 15 137 615.137
10 17 064 617.064
    613.564   615.137 "The first summer when you go to "
00:10:17.064 --> 0

14 24 048 864.048
00:14:24.048 --> 00:14:27.074
14 24 048 864.048
14 27 074 867.074
00:14:27.074 --> 00:14:28.948
14 27 074 867.074
14 28 948 868.948
00:14:28.948 --> 00:14:30.857
14 28 948 868.948
14 30 857 870.857
    854.311   868.948 "And these kids are extraordinary children, and I think  many people have this stereotype that Spelling Bee kids  are verbal geniuses and the ones who win  the Spelling Bee are sort of more genius-like  than the ones who don't win the Spelling Bee. "
00:14:30.857 --> 00:14:34.128
14 30 857 870.857
14 34 128 874.128
00:14:34.128 --> 00:14:38.796
14 34 128 874.128
14 38 796 878.796
    868.948   870.857 "So I asked the director if that were true and she said, "
00:14:38.796 --> 00:14:43.112
14 38 796 878.796
14 43 112 883.112
00:14:43.112 --> 00:14:46.045
14 43 112 883.112
14 46 045 886.045
00:14:46.045 --> 00:14:50.547
14 46 045 886.045
14 50 547 890.547
00:14:50.547 --> 00:14:53.728
14 50 547 890.547
14 53 728 893.728
00:14:53.728 --> 00:14:56.240
14 5

00:17:53.586 --> 00:17:55.940
17 53 586 1073.586
17 55 940 1075.940
00:17:55.940 --> 00:18:00.024
17 55 940 1075.940
18 0 024 1080.024
00:18:00.024 --> 00:18:02.936
18 0 024 1080.024
18 2 936 1082.936
00:18:02.936 --> 00:18:05.489
18 2 936 1082.936
18 5 489 1085.489
00:18:05.489 --> 00:18:08.530
18 5 489 1085.489
18 8 530 1088.530
    1070.433   1085.489 "If you look at early films of people that we all love --  maybe you love Will Smith -- I do --  maybe you love Matt Dillon or Rob Lowe,  take any Academy Award winning actor and  go watch one of their first films. "
00:18:08.530 --> 00:18:10.459
18 8 530 1088.530
18 10 459 1090.459
    1085.489   1088.530 "More likely than not it was terrible. "
00:18:10.459 --> 00:18:13.133
18 10 459 1090.459
18 13 133 1093.133
00:18:13.133 --> 00:18:14.855
18 13 133 1093.133
18 14 855 1094.855
00:18:14.855 --> 00:18:16.790
18 14 855 1094.855
18 16 790 1096.790
    1088.530   1094.855 "So the interesting thing is what makes somebody  have a terrible 